In [1]:
import torch as t
from torch import nn

from torchvision.datasets import MNIST
from torchvision import transforms as T
from torch.utils.data import DataLoader
from torch.autograd import Variable as V

import numpy as np

In [2]:
# Device configuration
device = t.device('cuda:0' if t.cuda.is_available() else 'cpu')

In [3]:
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [4]:
transforms = T.Compose([T.ToTensor()]) # T.ToTrensor() is a must

In [5]:
data_train = MNIST("./dataset/mnist-dataset/",
              train=True,
              transform=transforms,
              target_transform=None,
              download=False)

In [6]:
data_test = MNIST("./dataset/mnist-dataset/",
                  train=False,
                  transform=transforms,
                  target_transform=None,
                  download=False)

In [7]:
train_loader = DataLoader(data_train, 
                              batch_size=batch_size,
                              shuffle=True)
test_loader = DataLoader(data_test, 
                             batch_size=batch_size,
                             shuffle=True)

In [8]:
class MNIST_NET(nn.Module):
    '''
    Build a classification net for MNIST.
    '''
    def __init__(self):
        super(MNIST_NET, self).__init__()
        self.net_1 = nn.Sequential(nn.Conv2d(1, 8, 3, stride=1, padding=1),
                                 nn.ReLU(),
                                 nn.MaxPool2d(kernel_size=2))
        self.net_2 = nn.Sequential(nn.Conv2d(8, 32, 3, stride=1, padding=1),
                                  nn.ReLU(),
                                  nn.MaxPool2d(kernel_size=2))
        self.net_3 = nn.Sequential(nn.Conv2d(32, 64, 3, stride=1, padding=1),
                                  nn.ReLU(),
                                  nn.MaxPool2d(kernel_size=2))
        self.net_4 = nn.Sequential(nn.Linear(64*3*3, 100),
                                  nn.Linear(100,num_classes))
    def forward(self, input):
        x = self.net_1(input)
        x = self.net_2(x)
        x = self.net_3(x)
        x = x.view(-1, 64*3*3)
        x = self.net_4(x)
        return x

In [9]:
model = MNIST_NET()

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = t.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with t.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = t.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
t.save(model.state_dict(), 'model.ckpt')

Epoch [1/5], Step [100/600], Loss: 0.3141
Epoch [1/5], Step [200/600], Loss: 0.1373
Epoch [1/5], Step [300/600], Loss: 0.0504
Epoch [1/5], Step [400/600], Loss: 0.1073
Epoch [1/5], Step [500/600], Loss: 0.1669
Epoch [1/5], Step [600/600], Loss: 0.0682
Epoch [2/5], Step [100/600], Loss: 0.1167
Epoch [2/5], Step [200/600], Loss: 0.0587
Epoch [2/5], Step [300/600], Loss: 0.0211
Epoch [2/5], Step [400/600], Loss: 0.0290
Epoch [2/5], Step [500/600], Loss: 0.0560
Epoch [2/5], Step [600/600], Loss: 0.0296
Epoch [3/5], Step [100/600], Loss: 0.0758
Epoch [3/5], Step [200/600], Loss: 0.0311
Epoch [3/5], Step [300/600], Loss: 0.0597
Epoch [3/5], Step [400/600], Loss: 0.0247
Epoch [3/5], Step [500/600], Loss: 0.0256
Epoch [3/5], Step [600/600], Loss: 0.0600
Epoch [4/5], Step [100/600], Loss: 0.0078
Epoch [4/5], Step [200/600], Loss: 0.0050
Epoch [4/5], Step [300/600], Loss: 0.0150
Epoch [4/5], Step [400/600], Loss: 0.0724
Epoch [4/5], Step [500/600], Loss: 0.0076
Epoch [4/5], Step [600/600], Loss: